✅ Imports successful


URL: https://a002-ceqraccess.nyc.gov/ceqr/
Headers: {'content-type': 'application/x-www-form-urlencoded', 'referer': 'https://a002-ceqraccess.nyc.gov/ceqr/'}
Body: ctl00%24MainContent%24ddlBorough=Brooklyn&ctl00%24MainContent%24txtBlock=7061&ctl00%24MainContent%24txtLot=27

✅ Minimal request configured with only borough, block, and lot


Status Code: 200
Response Content Length: 32714 bytes

✅ Request successful!
📄 Response received (checking content...)

--- First 500 characters of response ---


<!DOCTYPE html>

<html lang="en">
<head><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta http-equiv="Content-Security-Policy" content="default-src &#39;self&#39;; script-src &#39;self&#39; &#39;unsafe-inline&#39; &#39;unsafe-eval&#39;; style-src &#39;self&#39; &#39;unsafe-inline&#39; &#39;unsafe-eval&#39;;  frame-src &#39;self&#39; https://winauth;" /><title>
	Project Search
</title><script src="/bundles/jQuery?v=5Br_kWrXaG2p_Z5FlR1md


In [ ]:
# Method 2: Get page first to establish session and extract VIEWSTATE
# This matches the working curl command approach

url = "https://a002-ceqraccess.nyc.gov/ceqr/"

# Step 1: GET the initial page to get cookies and VIEWSTATE
print("Step 1: Getting initial page to establish session...")
session = requests.Session()

# Headers for GET request
get_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": "https://a002-ceqraccess.nyc.gov/ceqr/"
}

initial_response = session.get(url, headers=get_headers, timeout=30)

if initial_response.status_code == 200:
    print(f"✅ Got initial page (cookies: {len(session.cookies)} cookies)")
    
    # Extract VIEWSTATE, VIEWSTATEGENERATOR, and EVENTVALIDATION
    soup = BeautifulSoup(initial_response.text, 'html.parser')
    
    viewstate_input = soup.find('input', {'id': '__VIEWSTATE'})
    viewstate_gen_input = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})
    eventval_input = soup.find('input', {'id': '__EVENTVALIDATION'})
    
    viewstate = viewstate_input['value'] if viewstate_input else ""
    viewstate_gen = viewstate_gen_input['value'] if viewstate_gen_input else "F2CE38DF"
    eventval = eventval_input['value'] if eventval_input else ""
    
    print(f"✅ Extracted VIEWSTATE ({len(viewstate)} chars)")
    print(f"✅ Extracted VIEWSTATEGENERATOR: {viewstate_gen}")
    print(f"✅ Extracted EVENTVALIDATION ({len(eventval)} chars)")
    
    # Step 2: Make POST request with cookies, VIEWSTATE, and search parameters
    print("\nStep 2: Making POST request with session cookies and VIEWSTATE...")
    
    # Search parameters (from curl command - Bronx, block 2756, no lot)
    # You can change these
    borough = "Bronx"
    block = "2756"
    lot = ""  # Empty lot
    
    # Build form data with VIEWSTATE
    form_data = {
        "__EVENTTARGET": "",
        "__EVENTARGUMENT": "",
        "__VIEWSTATE": viewstate,
        "__VIEWSTATEGENERATOR": viewstate_gen,
        "__SCROLLPOSITIONX": "0",
        "__SCROLLPOSITIONY": "0",
        "__VIEWSTATEENCRYPTED": "",
        "__EVENTVALIDATION": eventval,
        "ctl00$MainContent$txtKeyword": "",
        "ctl00$MainContent$ddlLeadAgency": "XYU@2!",
        "ctl00$MainContent$txtCeqrNumber": "",
        "ctl00$MainContent$txtProjectName": "",
        "ctl00$MainContent$ddlCommunityDistrict": "XYU@2!",
        "ctl00$MainContent$ddlBorough": borough,
        "ctl00$MainContent$txtBlock": block,
        "ctl00$MainContent$txtLot": lot,
        "ctl00$MainContent$btnSearch": " Search"
    }
    
    # Headers for POST request (matching curl)
    post_headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://a002-ceqraccess.nyc.gov",
        "Referer": "https://a002-ceqraccess.nyc.gov/ceqr/",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
        "Upgrade-Insecure-Requests": "1"
    }
    
    # Make POST request with session (cookies are automatically included)
    post_response = session.post(url, headers=post_headers, data=form_data, timeout=30)
    
    print(f"\nStatus Code: {post_response.status_code}")
    print(f"Response Length: {len(post_response.text)} chars")
    
    if post_response.status_code == 200:
        if 'grdSearchResults' in post_response.text or 'Search Results' in post_response.text:
            print("✅ Found search results!")
            
            # Store response for parsing
            response_with_cookies = post_response
            
            # Try to find CEQR numbers
            ceqr_pattern = r'\d{2}[A-Z]{3}\d{3}[A-Z]'
            ceqr_matches = re.findall(ceqr_pattern, post_response.text)
            if ceqr_matches:
                print(f"✅ Found CEQR numbers: {set(ceqr_matches)}")
        elif 'Error' in post_response.text:
            print("⚠️  Got error page")
        else:
            print("📄 Response received")
    else:
        print(f"❌ Request failed: {post_response.status_code}")
else:
    print(f"❌ Failed to get initial page: {initial_response.status_code}")


In [4]:
# Parse the HTML response to extract the table data
from bs4 import BeautifulSoup
import pandas as pd

# Use the response from cell 5 (or cell 9 if that worked better)
# For now, we'll re-run the request to get fresh data
try:
    # Make the request again to get the response
    response = requests.post(url, headers=headers, data=body, timeout=30)
    
    if response.status_code == 200:
        print("✅ Got response, parsing HTML...")
        
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the results table - look for table with id containing "grdSearchResults" or similar
        # The table might be in a grid or have a specific structure
        table = soup.find('table', {'id': lambda x: x and 'grdSearchResults' in x})
        
        if not table:
            # Try finding any table that might contain the results
            tables = soup.find_all('table')
            print(f"Found {len(tables)} tables in the page")
            
            # Look for table with headers containing "CEQR Number"
            for t in tables:
                headers_text = t.get_text()
                if 'CEQR Number' in headers_text or 'Project Name' in headers_text:
                    table = t
                    print(f"✅ Found results table")
                    break
        
        if table:
            print("✅ Found results table, extracting data...")
            
            # Extract table rows
            rows = table.find_all('tr')
            print(f"Found {len(rows)} rows in table")
            
            # Extract headers
            header_row = rows[0] if rows else None
            if header_row:
                headers_list = [th.get_text(strip=True) for th in header_row.find_all(['th', 'td'])]
                print(f"Headers: {headers_list}")
            
            # Extract data rows
            data_rows = []
            for row in rows[1:]:  # Skip header row
                cells = row.find_all(['td', 'th'])
                if cells:
                    row_data = [cell.get_text(strip=True) for cell in cells]
                    # Only add rows that have data (not empty)
                    if any(cell.strip() for cell in row_data):
                        data_rows.append(row_data)
            
            if data_rows:
                print(f"\n✅ Extracted {len(data_rows)} data rows")
                
                # Create DataFrame
                if header_row:
                    df = pd.DataFrame(data_rows, columns=headers_list[:len(data_rows[0])] if len(headers_list) >= len(data_rows[0]) else None)
                else:
                    df = pd.DataFrame(data_rows)
                
                # Display the results
                print("\n" + "=" * 80)
                print("CEQR Search Results:")
                print("=" * 80)
                print(df.to_string(index=False))
                
                # Also show as a more readable format
                print("\n" + "=" * 80)
                print("Results Summary:")
                print("=" * 80)
                for idx, row in df.iterrows():
                    print(f"\nRow {idx + 1}:")
                    for col in df.columns:
                        if pd.notna(row[col]) and str(row[col]).strip():
                            print(f"  {col}: {row[col]}")
            else:
                print("⚠️  No data rows found in table")
                # Show the table HTML structure for debugging
                print("\nTable HTML structure:")
                print(str(table)[:1000])
        else:
            print("⚠️  Could not find results table")
            print("\nSearching for CEQR-related content...")
            
            # Try to find CEQR numbers in the text
            ceqr_pattern = r'\d{2}[A-Z]{3}\d{3}[A-Z]'
            ceqr_matches = re.findall(ceqr_pattern, response.text)
            if ceqr_matches:
                print(f"Found CEQR numbers in text: {set(ceqr_matches)}")
            
            # Show a sample of the HTML to help debug
            print("\nSample HTML (first 2000 chars):")
            print(response.text)
    else:
        print(f"❌ Request failed: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error parsing response: {e}")
    import traceback
    traceback.print_exc()


✅ Got response, parsing HTML...
Found 4 tables in the page
⚠️  Could not find results table

Searching for CEQR-related content...

Sample HTML (first 2000 chars):


<!DOCTYPE html>

<html lang="en">
<head><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta http-equiv="Content-Security-Policy" content="default-src &#39;self&#39;; script-src &#39;self&#39; &#39;unsafe-inline&#39; &#39;unsafe-eval&#39;; style-src &#39;self&#39; &#39;unsafe-inline&#39; &#39;unsafe-eval&#39;;  frame-src &#39;self&#39; https://winauth;" /><title>
	Project Search
</title><script src="/bundles/jQuery?v=5Br_kWrXaG2p_Z5FlR1md42H9CV7IGPQPayuseC_3dM1"></script>
<script src="/bundles/modernizr?v=inCVuEFe6J4Q07A0AcRsbJic_UE5MwpRMNGcOtk94TE1"></script>
<link href="/Content/css?v=I4t_VAsuxLABXqcDpB75-Z7jI17GOShikmNppYLo5Zw1" rel="stylesheet"/>
<link href="../favicon.ico" rel="shortcut icon" type="image/x-icon" /></head>
<body>
    <form method="post" action="./" id="M